>Xgboost process data in its own format i.e.Dmatrix<br>
>CatBoot(Categorical Boosting) from company Yandex<br>
>>cat boost works withput hot encoding(without creating dummies)

In [41]:
from catboost import CatBoostClassifier,CatBoostRegressor

In [2]:
import pandas as pd
import numpy as np

In [3]:
#hr = pd.read_csv("/Users/darshmac/Documents/cdac/For DBDA/PML/Cases/human-resources-analytics/HR_comma_sep.csv")
hr = pd.read_csv("/kaggle/input/hr-comma-sep/HR_comma_sep.csv")
hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [37]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14995 entries, 0 to 14994
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14995 non-null  float64
 1   last_evaluation        14995 non-null  float64
 2   number_project         14995 non-null  int64  
 3   average_montly_hours   14995 non-null  int64  
 4   time_spend_company     14995 non-null  int64  
 5   Work_accident          14995 non-null  int64  
 6   left                   14995 non-null  int64  
 7   promotion_last_5years  14995 non-null  int64  
 8   Department             14995 non-null  object 
 9   salary                 14995 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [30]:
data_types = hr.dtypes == object
data_types[data_types == True]

Department    True
salary        True
dtype: bool

In [21]:
X = hr.drop('left',axis=1)
y = hr['left']

In [22]:
from sklearn.model_selection import StratifiedKFold,GridSearchCV

In [23]:
kflod = StratifiedKFold(n_splits=5, shuffle=True,random_state=23)

In [33]:
cbc = CatBoostClassifier(random_state=23,cat_features=['Department', "salary"])

In [38]:
cbgm = {'n_estimators':[25,50,75,100],
             'learning_rate':np.linspace(0.001,0.7,5),
             'max_depth':[2,3,5]}
cbgcv = GridSearchCV(cbc,param_grid=cbgm,cv=kflod,verbose=3,scoring='neg_log_loss')

In [39]:
%%time
cbgcv.fit(X,y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
0:	learn: 0.6925319	total: 3.14ms	remaining: 75.4ms
1:	learn: 0.6919303	total: 5.79ms	remaining: 66.6ms
2:	learn: 0.6913299	total: 8.04ms	remaining: 58.9ms
3:	learn: 0.6907183	total: 10.5ms	remaining: 54.9ms
4:	learn: 0.6901078	total: 12.7ms	remaining: 50.7ms
5:	learn: 0.6894985	total: 14.8ms	remaining: 46.9ms
6:	learn: 0.6888905	total: 17.2ms	remaining: 44.3ms
7:	learn: 0.6882960	total: 19.5ms	remaining: 41.4ms
8:	learn: 0.6876921	total: 21.7ms	remaining: 38.6ms
9:	learn: 0.6870876	total: 24.1ms	remaining: 36.2ms
10:	learn: 0.6864844	total: 26.7ms	remaining: 34ms
11:	learn: 0.6858840	total: 29.2ms	remaining: 31.6ms
12:	learn: 0.6852955	total: 32ms	remaining: 29.6ms
13:	learn: 0.6846975	total: 34.4ms	remaining: 27ms
14:	learn: 0.6841006	total: 36.8ms	remaining: 24.5ms
15:	learn: 0.6835155	total: 39.1ms	remaining: 22ms
16:	learn: 0.6829328	total: 41.5ms	remaining: 19.6ms
17:	learn: 0.6823500	total: 43.9ms	remaining: 17.1ms
18

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=<catboost.core.CatBoostClassifier object at 0x7a96e8c4f370>,
             param_grid={'learning_rate': array([0.001  , 0.17575, 0.3505 , 0.52525, 0.7    ]),
                         'max_depth': [2, 3, 5],
                         'n_estimators': [25, 50, 75, 100]},
             scoring='neg_log_loss', verbose=3)

In [40]:
print(cbgcv.best_params_)
print(cbgcv.best_score_)

{'learning_rate': 0.7, 'max_depth': 5, 'n_estimators': 100}
-0.07121065117148208


In [42]:
boss = pd.read_csv("/kaggle/input/boston/Boston.csv")
boss

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [44]:
X = boss.drop('medv',axis=1)
y = boss['medv']

In [48]:
from sklearn.model_selection import KFold,GridSearchCV
kflod = KFold(n_splits=5, shuffle=True,random_state=23)

In [49]:
cbr = CatBoostRegressor(random_state=23)

In [50]:
cbgr = {'n_estimators':[25,50,75,100],
             'learning_rate':np.linspace(0.001,0.7,5),
             'max_depth':[2,3,5]}
cbrgcv = GridSearchCV(cbr,param_grid=cbgr,cv=kflod,verbose=3)

In [51]:
%%time
cbrgcv.fit(X,y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
0:	learn: 9.1599871	total: 636us	remaining: 15.3ms
1:	learn: 9.1545304	total: 1.06ms	remaining: 12.2ms
2:	learn: 9.1491074	total: 1.46ms	remaining: 10.7ms
3:	learn: 9.1436422	total: 1.84ms	remaining: 9.65ms
4:	learn: 9.1395390	total: 2.2ms	remaining: 8.8ms
5:	learn: 9.1348773	total: 2.58ms	remaining: 8.18ms
6:	learn: 9.1293910	total: 2.96ms	remaining: 7.6ms
7:	learn: 9.1252787	total: 3.4ms	remaining: 7.22ms
8:	learn: 9.1204973	total: 3.79ms	remaining: 6.74ms
9:	learn: 9.1158892	total: 13.3ms	remaining: 19.9ms
10:	learn: 9.1109262	total: 13.6ms	remaining: 17.3ms
11:	learn: 9.1056265	total: 14ms	remaining: 15.2ms
12:	learn: 9.1003482	total: 14.4ms	remaining: 13.3ms
13:	learn: 9.0964439	total: 14.7ms	remaining: 11.6ms
14:	learn: 9.0909842	total: 15.1ms	remaining: 10.1ms
15:	learn: 9.0867554	total: 15.4ms	remaining: 8.67ms
16:	learn: 9.0824548	total: 15.7ms	remaining: 7.41ms
17:	learn: 9.0771118	total: 16.1ms	remaining: 6.24ms
1

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x7a969a61c8e0>,
             param_grid={'learning_rate': array([0.001  , 0.17575, 0.3505 , 0.52525, 0.7    ]),
                         'max_depth': [2, 3, 5],
                         'n_estimators': [25, 50, 75, 100]},
             verbose=3)

In [52]:
print(cbrgcv.best_params_)
print(cbrgcv.best_score_)

{'learning_rate': 0.17575, 'max_depth': 5, 'n_estimators': 100}
0.8809874524169409
